In [ ]:
import sys
!{sys.executable} -m pip install opentrons

Cycle 0 Protocol Simulation

In [1]:
from opentrons import simulate
protocol = simulate.get_protocol_api("2.7")  
protocol.home()

source = protocol.load_labware("nest_12_reservoir_15ml", "1") #reservoir containing modified (A1) and unmodified E.coli (A2)
reservoir = protocol.load_labware("nest_12_reservoir_15ml", "7") #reservoir containing mutagen + growth medias (A1-A4) and broth (A5)
tiprack_1 = protocol.load_labware("opentrons_96_tiprack_300ul", "5")
tiprack_2 = protocol.load_labware("opentrons_96_tiprack_300ul", "8")
tiprack_3 = protocol.load_labware("opentrons_96_tiprack_300ul", "9")
tiprack_4 = protocol.load_labware("opentrons_96_tiprack_20ul", "6")
p300 = protocol.load_instrument("p300_multi_gen2", "left", tip_racks=[tiprack_1, tiprack_2, tiprack_3])
p20 = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tiprack_4])
temp_mod = protocol.load_module("temperature module", "4")
dest = temp_mod.load_labware("corning_96_wellplate_360ul_flat") #plate to be incubated
test = protocol.load_labware("corning_96_wellplate_360ul_flat", "3") #test plate for testing fluorescence

#transferring 5ul of modified E.coli to each well of columns 1 - 10 of the destination plate
p20.pick_up_tip()
for i in range(10):
    p20.transfer(5, source.wells("A1"), dest.columns()[i], new_tip = "never")
p20.drop_tip()

#transferring 5 ul of non-modified E.coli to  each well of column 11 of the destination plate
p20.pick_up_tip()
p20.transfer(5, source.wells("A2"), dest.columns()[10], new_tip = "never")
p20.drop_tip()

#adding components to encoruage growth to all wells of the destination plate
for i in range(4):
    p300.pick_up_tip()
    p300.distribute(20, reservoir.wells()[i], dest.wells(), new_tip = "never")
    p300.drop_tip()

#adding "broth" to all wells and mixing - here tips are changed every time to ensure no contamination
for i in range(12):
    p300.pick_up_tip()
    p300.transfer(115, reservoir.wells("A5"), dest.columns()[i], mix_after = (3,100), new_tip = "never")
    p300.drop_tip()

temp_mod.set_temperature(35) #set temperature of the heating block to 35oC

protocol.delay(minutes=240) #once temperature is reached, the destination plate will incubate for 4 hours
temp_mod.set_temperature(24) #force cools the heating block back to room temperature before continuing

#takes a 100 ul sample of each well and transfers it to the test plate to for fluorescence testing
for i in range(12):
    p300.pick_up_tip()
    p300.transfer(100, dest.columns()[i], test.columns()[i], new_tip = "never")
    p300.drop_tip()
    
for line in protocol.commands():
    print(line)

C:\Users\mde13\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\mde13\.opentrons\deck_calibration.json not found. Loading defaults


Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 6
Transferring 5.0 from A1 of NEST 12 Well Reservoir 15 mL on 1 to A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4
Aspirating 5.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 1 at 7.56 uL/sec
Dispensing 5.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 1 at 7.56 uL/sec
Dispensing 5.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 1 at 7.56 uL/sec
Dispensing 5.0 uL into C1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 1 at 7.56 uL/sec
Dispensing 5.0 uL into D1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from A1 of NEST 12 Well Reservoir 

Script (1) - Fluorescence Data Processing

In [2]:
print("To copy the file path, select the file, use 'shift + right click', and select 'copy file path'.\nPlease make sure to remove quotation marks or file will not be recognised!\n")

from openpyxl import load_workbook
wb = load_workbook(filename = input("Please enter file path = "))
sheet = wb.active

#code reads the specified excel file - an ouptut file from the omega, finds and collates the well references that show the highest flourescence
columns_excel = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
rows_opentron = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
columns_opentron =[]

well_coord =[]
well_value = []

for i in range(len(columns_excel)):
    index = str(columns_excel[i])
    for n in range(len(rows_opentron)):
        well_value.append(sheet[str(index+str(n+15))].value)
        well_coord.append(str(rows_opentron[n] + str(i+1)))
    
wells = {well_coord[i]: well_value[i] for i in range(len(well_coord))}

wells_to_pop = wells.copy()

highest_wells = []
for i in range(int(input("\nHow many wells would you like to extract? (10 is advised) = "))): #user input to select how many wells
    highest = max(wells_to_pop, key = wells.get)
    highest_wells.append(highest)
    wells_to_pop.pop(highest) #arranges the highest fluorescence well references in a list from highest to lowest

keys = [] #creates the index keys for the dictionary
for i in range(0, len(highest_wells)):
    keys.append(i)
    
dic = dict(zip(highest_wells, keys)) #zips the well references and keys into a dictionary

print("\nDictionary Input for Well Extraction Protocol = ", dic )
print("\nPlease simply copy this dictionary into the space following 'wells =' in the Well Extraction Protocol")

To copy the file path, select the file, use 'shift + right click', and select 'copy file path'.
Please make sure to remove quotation marks or file will not be recognised!

Please enter file path = C:\Users\mde13\OneDrive - Imperial College London\2020-2021 - MRes Synthetic & Systems Biology\Tron Weiss - Serial Dilution + New Protocol (All docs)\New Protocol\Simulation\Final - mock fluprescence data.xlsx

How many wells would you like to extract? (10 is advised) = 10

Dictionary Input for Well Extraction Protocol =  {'F11': 0, 'D9': 1, 'F10': 2, 'D11': 3, 'F1': 4, 'H5': 5, 'D8': 6, 'H1': 7, 'E3': 8, 'D2': 9}

Please simply copy this dictionary into the space following 'wells =' in the Well Extraction Protocol


Script (2) - New Protocol Generation

In [ ]:
New_Cycle_Protocol = """from opentrons import protocol_api

metadata = {'apiLevel': '2.7'}

def run(protocol: protocol_api.ProtocolContext):
    source = protocol.load_labware("corning_96_wellplate_360ul_flat", "1")
    reservoir = protocol.load_labware("nest_12_reservoir_15ml", "7")
    tiprack_1 = protocol.load_labware("opentrons_96_tiprack_300ul", "5")
    tiprack_2 = protocol.load_labware("opentrons_96_tiprack_300ul", "8")
    tiprack_3 = protocol.load_labware("opentrons_96_tiprack_300ul", "9")
    tiprack_4 = protocol.load_labware("opentrons_96_tiprack_20ul", "6")
    p300 = protocol.load_instrument("p300_multi_gen2", "left", tip_racks=[tiprack_1, tiprack_2, tiprack_3])
    p20 = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tiprack_4])
    temp_mod = protocol.load_module("temperature module", "4")
    dest = temp_mod.load_labware("corning_96_wellplate_360ul_flat")
    test = protocol.load_labware("corning_96_wellplate_360ul_flat", "3")

    wells =   #<-- Paste the dictionary from the above code here!

    j = 0

    for i in wells:
        p20.starting_tip = tiprack_4.well("C1")
        p20.pick_up_tip()
        p20.transfer(5, source.wells_by_name()[i], dest.columns(j), new_tip = "never")
        j += 1
        p20.drop_tip()

    p20.transfer(5, source.columns_by_name()["11"], dest.columns_by_name()["11"], new_tip = "always")

    for i in range(4):
        p300.pick_up_tip()
        p300.distribute(20, reservoir.wells()[i], dest.wells(), new_tip = "never")
        p300.drop_tip()

    for i in range(12):
        p300.pick_up_tip()
        p300.transfer(115, reservoir.wells("A5"), dest.columns()[i], mix_after = (3,100), new_tip = "never")
        p300.drop_tip()

    temp_mod.set_temperature(35)

    protocol.delay(minutes=240)
    temp_mod.set_temperature(24) #sets temperature back to room

    for i in range(12):
        p300.pick_up_tip()
        p300.transfer(100, dest.columns()[i], test.columns()[i], new_tip = "never")
        p300.drop_tip()"""

file = open("New_Protocol.py",'w')
new = file.write(New_Cycle_Protocol)
file.close()

New_Protocol (Simulation) - this is a simulation using the code above to generate a protocol with a dictionary obtained by

"mock fluprescence data"

In [3]:
from opentrons import simulate
protocol = simulate.get_protocol_api("2.7")  
protocol.home()

source = protocol.load_labware("corning_96_wellplate_360ul_flat", "1")
reservoir = protocol.load_labware("nest_12_reservoir_15ml", "7")
tiprack_1 = protocol.load_labware("opentrons_96_tiprack_300ul", "5")
tiprack_2 = protocol.load_labware("opentrons_96_tiprack_300ul", "8")
tiprack_3 = protocol.load_labware("opentrons_96_tiprack_300ul", "9")
tiprack_4 = protocol.load_labware("opentrons_96_tiprack_20ul", "6")
p300 = protocol.load_instrument("p300_multi_gen2", "left", tip_racks=[tiprack_1, tiprack_2, tiprack_3])
p20 = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tiprack_4])
temp_mod = protocol.load_module("temperature module", "4")
dest = temp_mod.load_labware("corning_96_wellplate_360ul_flat")
test = protocol.load_labware("corning_96_wellplate_360ul_flat", "3")

wells = {'F11': 0, 'D9': 1, 'F10': 2, 'D11': 3, 'F1': 4, 'H5': 5, 'D8': 6, 'H1': 7, 'E3': 8, 'D2': 9} #<-- Paste the dictionary from the above code here!

#transferring 5ul of E.coli from each of the 10 highest fluorescing wells in the original "dest" plate (the new source plate) to the new dest plat
j = 0
for i in wells:
    p20.starting_tip = tiprack_4.well("C1")
    p20.pick_up_tip()
    p20.transfer(5, source.wells_by_name()[i], dest.columns(j), new_tip = "never")
    j += 1
    p20.drop_tip()

#transferring controls from the original "dest" plate (the new source plate) to the new "dest" plate
p20.transfer(5, source.columns_by_name()["11"], dest.columns_by_name()["11"], new_tip = "always")

#adding components to encourage growth to all wells of the destination plate
for i in range(4):
    p300.pick_up_tip()
    p300.distribute(20, reservoir.wells()[i], dest.wells(), new_tip = "never")
    p300.drop_tip()

#adding "broth" to all wells and mixing - here tips are changed every time to ensure no contamination
for i in range(12):
    p300.pick_up_tip()
    p300.transfer(115, reservoir.wells("A5"), dest.columns()[i], mix_after = (3,100), new_tip = "never")
    p300.drop_tip()

temp_mod.set_temperature(35) #set temperature of the heating block to 35oC

protocol.delay(minutes=240) #once temperature is reached, the destination plate will incubate for 4 hours
temp_mod.set_temperature(24) #force cools the heating block back to room temperature before continuing

#takes a 100 ul sample of each well and transfers it to the test plate to for fluorescence testing
for i in range(12):
    p300.pick_up_tip()
    p300.transfer(100, dest.columns()[i], test.columns()[i], new_tip = "never")
    p300.drop_tip()

for line in protocol.commands():
    print(line)

C:\Users\mde13\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\mde13\.opentrons\deck_calibration.json not found. Loading defaults


Picking up tip from C1 of Opentrons 96 Tip Rack 20 µL on 6
Transferring 5.0 from F11 of Corning 96 Well Plate 360 µL Flat on 1 to A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4
Aspirating 5.0 uL from F11 of Corning 96 Well Plate 360 µL Flat on 1 at 7.56 uL/sec
Dispensing 5.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from F11 of Corning 96 Well Plate 360 µL Flat on 1 at 7.56 uL/sec
Dispensing 5.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from F11 of Corning 96 Well Plate 360 µL Flat on 1 at 7.56 uL/sec
Dispensing 5.0 uL into C1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from F11 of Corning 96 Well Plate 360 µL Flat on 1 at 7.56 uL/sec
Dispensing 5.0 uL into D1 of Corning 96 Well Plate 360 µL Flat on Temperature Module GEN1 on 4 at 7.56 uL/sec
Aspirating 5.0 uL from